# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [635]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Generate the mathematical model

In [641]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [642]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for the Farmer's Problem

**Sets:**

* I = {Wheat, Corn, Sugar beet}: Set of crops

**Parameters:**

* A = 500: Total acres of land available
* D<sub>wheat</sub> = 200: Demand for wheat for cattle feed (tons)
* D<sub>corn</sub> = 240: Demand for corn for cattle feed (tons)
* SP<sub>wheat</sub> = 170: Selling price per ton of wheat ($)
* SP<sub>corn</sub> = 150: Selling price per ton of corn ($)
* SP<sub>sugar beet</sub> = 36: Selling price per ton of sugar beet within quota ($)
* SP<sub>sugar beet excess</sub> = 10: Selling price per ton of sugar beet exceeding quota ($)
* PP<sub>wheat</sub> = 170 * 1.4 = 238: Purchase price per ton of wheat ($)
* PP<sub>corn</sub> = 150 * 1.4 = 210: Purchase price per ton of corn ($)
* Q<sub>sugar beet</sub> = 6000: Quota for sugar beet production (tons)
* Y<sub>wheat</sub> = 2.5: Yield of wheat per acre (tons/acre)
* Y<sub>corn</sub> = 3: Yield of corn per acre (tons/acre)
* Y<sub>sugar beet</sub> = 20: Yield of sugar beet per acre (tons/acre)
* C<sub>wheat</sub> = 150: Planting cost per acre of wheat ($)
* C<sub>corn</sub> = 230: Planting cost per acre of corn ($)
* C<sub>sugar beet</sub> = 260: Planting cost per acre of sugar beet ($)

**Decision Variables:**

* X<sub>i</sub>: Acres of land devoted to crop i ∈ I
* P<sub>i</sub>: Tons of crop i ∈ I purchased 
* S<sub>i</sub>: Tons of crop i ∈ I sold within quota
* S<sub>excess</sub>: Tons of sugar beet sold exceeding quota

**Objective Function:** Minimize total cost

```
Minimize: Σ<sub>i∈I</sub> (C<sub>i</sub> * X<sub>i</sub> + PP<sub>i</sub> * P<sub>i</sub> - SP<sub>i</sub> * S<sub>i</sub>) - SP<sub>sugar beet excess</sub> * S<sub>excess</sub> 
```

**Constraints:**

1. **Land availability:**
   ```
   Σ<sub>i∈I</sub> X<sub>i</sub> ≤ A
   ```
2. **Production and purchase to meet demand:**
   ```
   Y<sub>wheat</sub> * X<sub>wheat</sub> + P<sub>wheat</sub> ≥ D<sub>wheat</sub>
   Y<sub>corn</sub> * X<sub>corn</sub> + P<sub>corn</sub> ≥ D<sub>corn</sub>
   ```
3. **Sales cannot exceed production and purchase:**
   ```
   S<sub>wheat</sub> ≤ Y<sub>wheat</sub> * X<sub>wheat</sub> + P<sub>wheat</sub> - D<sub>wheat</sub>
   S<sub>corn</sub> ≤ Y<sub>corn</sub> * X<sub>corn</sub> + P<sub>corn</sub> - D<sub>corn</sub>
   S<sub>sugar beet</sub> ≤ Y<sub>sugar beet</sub> * X<sub>sugar beet</sub>
   ```
4. **Sugar beet quota:**
   ```
   S<sub>sugar beet</sub> ≤ Q<sub>sugar beet</sub>
   ```
5. **Excess sugar beet sales:**
   ```
   S<sub>excess</sub> = Y<sub>sugar beet</sub> * X<sub>sugar beet</sub> - S<sub>sugar beet</sub>
   ```
6. **Non-negativity:**
   ```
   X<sub>i</sub>, P<sub>i</sub>, S<sub>i</sub>, S<sub>excess</sub> ≥ 0, ∀ i ∈ I
   ```

This model allows the farmer to determine the optimal allocation of land for each crop, considering the costs of planting, buying, and selling, as well as the production yield and demand for each product. The objective is to minimize the total cost while meeting the demand for cattle feed and respecting the sugar beet production quota. 


## 3. Generate the pyomo code

In [643]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [644]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define sets
I = ['Wheat', 'Corn', 'Sugar beet']

# Define parameters
A = 500
D = {'Wheat': 200, 'Corn': 240, 'Sugar beet': 0} # Note: We assume no demand for sugar beet itself, only selling 
SP = {'Wheat': 170, 'Corn': 150, 'Sugar beet': 36, 'Sugar beet excess': 10}
PP = {'Wheat': 238, 'Corn': 210, 'Sugar beet': 0} # Note: Assuming no purchasing of sugar beet
Q = {'Sugar beet': 6000}
Y = {'Wheat': 2.5, 'Corn': 3, 'Sugar beet': 20}
C = {'Wheat': 150, 'Corn': 230, 'Sugar beet': 260}

# Create model
model = pyo.ConcreteModel()

# Define variables
model.X = pyo.Var(I, domain=pyo.NonNegativeReals) # Acres of land devoted to each crop
model.P = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop purchased
model.S = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop sold within quota
model.S_excess = pyo.Var(domain=pyo.NonNegativeReals) # Tons of sugar beet sold exceeding quota

# Define objective function
def objective_rule(model):
  return sum(C[i] * model.X[i] + PP[i] * model.P[i] - SP[i] * model.S[i] for i in I) - SP['Sugar beet excess'] * model.S_excess
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
model.land_availability = pyo.Constraint(expr=sum(model.X[i] for i in I) <= A)

model.demand_wheat = pyo.Constraint(expr=Y['Wheat'] * model.X['Wheat'] + model.P['Wheat'] >= D['Wheat'])
model.demand_corn = pyo.Constraint(expr=Y['Corn'] * model.X['Corn'] + model.P['Corn'] >= D['Corn'])

model.sales_wheat = pyo.Constraint(expr=model.S['Wheat'] <= Y['Wheat'] * model.X['Wheat'] + model.P['Wheat'] - D['Wheat'])
model.sales_corn = pyo.Constraint(expr=model.S['Corn'] <= Y['Corn'] * model.X['Corn'] + model.P['Corn'] - D['Corn'])
model.sales_sugar_beet = pyo.Constraint(expr=model.S['Sugar beet'] <= Y['Sugar beet'] * model.X['Sugar beet'])

model.quota_sugar_beet = pyo.Constraint(expr=model.S['Sugar beet'] <= Q['Sugar beet'])

model.excess_sugar_beet = pyo.Constraint(expr=model.S_excess == Y['Sugar beet'] * model.X['Sugar beet'] - model.S['Sugar beet'])

# Solve the model
solver = pyo.SolverFactory('glpk') # You can choose a different solver if you prefer
solver.solve(model)

# Print results
print("Optimal solution:")
print(f"Objective value: ${model.objective():.2f}")
for i in I:
  print(f"Acres of {i}: {model.X[i]():.2f}")
  print(f"Tons of {i} purchased: {model.P[i]():.2f}")
  print(f"Tons of {i} sold: {model.S[i]():.2f}")
print(f"Tons of sugar beet sold exceeding quota: {model.S_excess():.2f}")
```

This code defines the sets, parameters, variables, objective function, and constraints of the mathematical optimization model for the farmer's problem. It then uses the Pyomo library to solve the model and print the optimal solution. This includes the optimal land allocation for each crop, the amount of each crop purchased and sold, and the amount of sugar beet sold exceeding the quota.


## 4. Input problem data and try running the generated code

In [648]:
import pyomo.environ as pyo

# Define sets
I = ['Wheat', 'Corn', 'Sugar beet']

# Define parameters
A = 500
D = {'Wheat': 200, 'Corn': 240, 'Sugar beet': 0} # Note: We assume no demand for sugar beet itself, only selling 
SP = {'Wheat': 170, 'Corn': 150, 'Sugar beet': 36, 'Sugar beet excess': 10}
PP = {'Wheat': 238, 'Corn': 210, 'Sugar beet': 0} # Note: Assuming no purchasing of sugar beet
Q = {'Sugar beet': 6000}
Y = {'Wheat': 2.5, 'Corn': 3, 'Sugar beet': 20}
C = {'Wheat': 150, 'Corn': 230, 'Sugar beet': 260}

# Create model
model = pyo.ConcreteModel()

# Define variables
model.X = pyo.Var(I, domain=pyo.NonNegativeReals) # Acres of land devoted to each crop
model.P = pyo.Var(I, domain=pyo.NonNegativeReals, initialize=0) # Tons of each crop purchased
model.S = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop sold within quota
model.S_excess = pyo.Var(domain=pyo.NonNegativeReals) # Tons of sugar beet sold exceeding quota

# Define objective function
def objective_rule(model):
    return sum(C[i] * model.X[i] + PP[i] * model.P[i] - SP[i] * model.S[i] for i in I) - SP['Sugar beet excess'] * model.S_excess
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
model.land_availability = pyo.Constraint(expr=sum(model.X[i] for i in I) <= A)

model.demand_wheat = pyo.Constraint(expr=Y['Wheat'] * model.X['Wheat'] + model.P['Wheat'] >= D['Wheat'])
model.demand_corn = pyo.Constraint(expr=Y['Corn'] * model.X['Corn'] + model.P['Corn'] >= D['Corn'])

model.sales_wheat = pyo.Constraint(expr=model.S['Wheat'] <= Y['Wheat'] * model.X['Wheat'] + model.P['Wheat'] - D['Wheat'])
model.sales_corn = pyo.Constraint(expr=model.S['Corn'] <= Y['Corn'] * model.X['Corn'] + model.P['Corn'] - D['Corn'])
model.sales_sugar_beet = pyo.Constraint(expr=model.S['Sugar beet'] <= Y['Sugar beet'] * model.X['Sugar beet'])

model.quota_sugar_beet = pyo.Constraint(expr=model.S['Sugar beet'] <= Q['Sugar beet'])

model.excess_sugar_beet = pyo.Constraint(expr=model.S_excess == Y['Sugar beet'] * model.X['Sugar beet'] - model.S['Sugar beet'])

# Solve the model
solver = pyo.SolverFactory('glpk') # You can choose a different solver if you prefer
solver.solve(model)

# Print results
print("Optimal solution:")
print(f"Objective value: ${model.objective():.2f}")
for i in I:
    print(f"Acres of {i}: {model.X[i]():.2f}")
    print(f"Tons of {i} purchased: {model.P[i]():.2f}")
    print(f"Tons of {i} sold: {model.S[i]():.2f}")
print(f"Tons of sugar beet sold exceeding quota: {model.S_excess():.2f}")

Optimal solution:
Objective value: $-118600.00
Acres of Wheat: 120.00
Tons of Wheat purchased: 0.00
Tons of Wheat sold: 100.00
Acres of Corn: 80.00
Tons of Corn purchased: 0.00
Tons of Corn sold: 0.00
Acres of Sugar beet: 300.00
Tons of Sugar beet purchased: 0.00
Tons of Sugar beet sold: 6000.00
Tons of sugar beet sold exceeding quota: 0.00


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [649]:
print(response.text)

## Mathematical Optimization Model for the Farmer's Problem

**Sets:**

* I = {Wheat, Corn, Sugar beet}: Set of crops

**Parameters:**

* A = 500: Total acres of land available
* D<sub>wheat</sub> = 200: Demand for wheat for cattle feed (tons)
* D<sub>corn</sub> = 240: Demand for corn for cattle feed (tons)
* SP<sub>wheat</sub> = 170: Selling price per ton of wheat ($)
* SP<sub>corn</sub> = 150: Selling price per ton of corn ($)
* SP<sub>sugar beet</sub> = 36: Selling price per ton of sugar beet within quota ($)
* SP<sub>sugar beet excess</sub> = 10: Selling price per ton of sugar beet exceeding quota ($)
* PP<sub>wheat</sub> = 170 * 1.4 = 238: Purchase price per ton of wheat ($)
* PP<sub>corn</sub> = 150 * 1.4 = 210: Purchase price per ton of corn ($)
* Q<sub>sugar beet</sub> = 6000: Quota for sugar beet production (tons)
* Y<sub>wheat</sub> = 2.5: Yield of wheat per acre (tons/acre)
* Y<sub>corn</sub> = 3: Yield of corn per acre (tons/acre)
* Y<sub>sugar beet</sub> = 20: Yield of sug

In [650]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define sets
I = ['Wheat', 'Corn', 'Sugar beet']

# Define parameters
A = 500
D = {'Wheat': 200, 'Corn': 240, 'Sugar beet': 0} # Note: We assume no demand for sugar beet itself, only selling 
SP = {'Wheat': 170, 'Corn': 150, 'Sugar beet': 36, 'Sugar beet excess': 10}
PP = {'Wheat': 238, 'Corn': 210, 'Sugar beet': 0} # Note: Assuming no purchasing of sugar beet
Q = {'Sugar beet': 6000}
Y = {'Wheat': 2.5, 'Corn': 3, 'Sugar beet': 20}
C = {'Wheat': 150, 'Corn': 230, 'Sugar beet': 260}

# Create model
model = pyo.ConcreteModel()

# Define variables
model.X = pyo.Var(I, domain=pyo.NonNegativeReals) # Acres of land devoted to each crop
model.P = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop purchased
model.S = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop sold within quota
model.S_excess = pyo.Var(domain=pyo.NonNegativeReals) # Tons of sugar beet sold exceeding quota

# Define objective function
def objective_rule(model):
  